<a href="https://colab.research.google.com/github/cksac/colab/blob/master/Tangent_Source_to_Source_Debuggable_Derivatives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tangent

* [Announcement](https://research.googleblog.com/2017/11/tangent-source-to-source-debuggable.html)
* [Github repo](https://github.com/google/tangent)

This notebook is just the examples from the `README` in executable form.

## Setup

We need to install `tangent` and any dependencies.

In [0]:
!pip install tangent

    100% |████████████████████████████████| 81kB 2.2MB/s 
    100% |████████████████████████████████| 163kB 3.0MB/s 
    100% |████████████████████████████████| 44.7MB 26kB/s 
    100% |████████████████████████████████| 81kB 8.7MB/s 
    100% |████████████████████████████████| 3.0MB 364kB/s 
  Running setup.py bdist_wheel for tangent ... - done
  Stored in directory: /content/.cache/pip/wheels/f0/ae/c2/5b094a427f1a1b31c32b94df26678b3efcfe7fedddd2ab4181
  Running setup.py bdist_wheel for autograd ... - done
  Stored in directory: /content/.cache/pip/wheels/ea/d4/2d/e7d74fb5d727853026c54f23fa96f0b8defefc7d57dcaca0aa
  Running setup.py bdist_wheel for gast ... - done
  Stored in directory: /content/.cache/pip/wheels/8e/fa/d6/77dd17d18ea23fd7b860e02623d27c1be451521af40dd4a13e
  Running setup.py bdist_wheel for absl-py ... -

 done
  Stored in directory: /content/.cache/pip/wheels/04/f5/7c/5d4eab10ddf87dec875016e74ba289d87270a90fb2662a76fc
  Running setup.py bdist_wheel for termcolor ... - done
  Stored in directory: /content/.cache/pip/wheels/de/f7/bf/1bcac7bf30549e6a4957382e2ecab04c88e513117207067b03
Successfully built tangent autograd gast absl-py termcolor


# Basic examples

In [0]:
def f(x):
  a = x * x
  b = x * a
  c = a + b
  return c

In [0]:
f(3.0)

36.0

In [0]:
import tangent
df = tangent.grad(f)

In [0]:
df(3.0)

33.0

Re-executing the cell below will also show the source for `df`:

In [0]:
df??

## Tangent and TF Eager

In [0]:
import tangent
import tensorflow as tf

def f(W,x):
  h1 = tf.matmul(x,W)
  h2 = tf.tanh(h1)
  out = tf.reduce_sum(h2)
  return out

dfdW = tangent.grad(f, verbose=1)

def dfdW(W, x, bout=1.0):
    h1 = tf.matmul(x, W)
    h2 = tf.tanh(h1)
    out = tf.reduce_sum(h2)
    assert tangent.shapes_match(out, bout
        ), 'Shape mismatch between return value (%s) and seed derivative (%s)' % (
        numpy.shape(out), numpy.shape(bout))

    # Grad of: out = tf.reduce_sum(h2)
    _bh2 = tangent.unreduce(bout, tangent.shape_as_list(h2), None, False)
    bh2 = _bh2

    # Grad of: h2 = tf.tanh(h1)
    _h2 = h2
    _bh1 = bh2 * (1 - _h2 * _h2)
    bh1 = _bh1

    # Grad of: h1 = tf.matmul(x, W)
    _bW = tangent.matmul_adjoint_y(bh1, x, W, False, False)
    bW = _bW
    return bW



## Tangent and subroutines

In [0]:
def f(x):
  a = _mul(x, x)
  return a

def _mul(x, y):
  out = x * y
  return out

import tangent
tangent.grad(f, verbose=1)

def dfdx(x, ba=1.0):
    # Initialize the tape
    _stack = tangent.Stack()
    _substack = tangent.Stack()
    tangent.push_stack(_stack, _substack, '_d5d22c0d')
    a = pri__mulxy(_substack, x, x)
    assert tangent.shapes_match(a, ba
        ), 'Shape mismatch between return value (%s) and seed derivative (%s)' % (
        numpy.shape(a), numpy.shape(ba))

    # Grad of: a = _mul(x, x)
    _substack = tangent.pop_stack(_stack, '_d5d22c0d')
    dxs = _d_muldxy(_substack, ba, x, x)
    _bx = dxs[0]
    _bx2 = dxs[1]
    bx = _bx
    bx = tangent.add_grad(bx, _bx2)
    return bx


def pri__mulxy(_stack, x, y):
    out = x * y
    result = out
    tangent.push(_stack, result, '_a1151cb1')
    return out


def _d_muldxy(_stack, bout, x, y):
    result = tangent.pop(_stack, '_a1151cb1')

    # Grad of: out = x * y
    _bx = tangent.unbroadcast(bout * y, x)
    _by = tangent.unbroadcast(bout * x, y)
    bx = _bx
    by = _by
    return bx, by, result



<function tangent_21c9.dfdx>

## Tangent and Control Flow

In [0]:
def f(x):
  if x > 0:
    a = x ** 2.0
  else:
    a = -x
  out = a * a
  return out

tangent.grad(f, verbose=True)

def dfdx(x, bout=1.0):
    # Initialize the tape
    _stack = tangent.Stack()

    # Beginning of forward pass
    cond = x > 0
    if cond:
        a = x ** 2.0
    else:
        a = -x
    tangent.push(_stack, cond, '_086298b7')
    out = a * a
    assert tangent.shapes_match(out, bout
        ), 'Shape mismatch between return value (%s) and seed derivative (%s)' % (
        numpy.shape(out), numpy.shape(bout))

    # Grad of: out = a * a
    _ba = tangent.unbroadcast(bout * a, a)
    _ba2 = tangent.unbroadcast(bout * a, a)
    ba = _ba
    ba = tangent.add_grad(ba, _ba2)
    cond = tangent.pop(_stack, '_086298b7')
    if cond:
        # Grad of: a = x ** 2.0
        _bx = 2.0 * x * ba
        bx = _bx
    else:
        # Grad of: a = -x
        _bx2 = -ba
        bx = _bx2
    return bx



<function tangent_961c.dfdx>

## Tangent and loops

In [0]:
def f(x):
  for i in range(3):
    x = x ** 0.5
  return x

tangent.grad(f, verbose=True)

def dfdx(x, bx=1.0):
    # Initialize the tape
    _stack = tangent.Stack()

    # Beginning of forward pass
    i2 = 0
    for i in range(3):
        i2 += 1
        tangent.push(_stack, x, '_4e4bfb98')
        x = x ** 0.5
    tangent.push(_stack, i2, '_cffbe358')
    assert tangent.shapes_match(x, bx
        ), 'Shape mismatch between return value (%s) and seed derivative (%s)' % (
        numpy.shape(x), numpy.shape(bx))

    # Beginning of backward pass
    i2 = tangent.pop(_stack, '_cffbe358')
    for _ in range(i2):
        # Grad of: x = x ** 0.5
        _x = x
        x = tangent.pop(_stack, '_4e4bfb98')
        _bx = 0.5 * x ** -0.5 * bx
        bx = tangent.init_grad(x, allow_lazy_initializer=True)
        bx = tangent.add_grad(bx, _bx)
    return bx



<function tangent_61cd.dfdx>

# Tangent and Debugging

In [0]:
import tangent
import pdb

def f(x):
  a = x * x
  b = x * a
  c = a + b
  
  pdb.set_trace()
  return c

tangent.grad(f, verbose=True)

def dfdx(x, bc=1.0):
    a = x * x
    b = x * a
    c = a + b
    pdb.set_trace()
    assert tangent.shapes_match(c, bc
        ), 'Shape mismatch between return value (%s) and seed derivative (%s)' % (
        numpy.shape(c), numpy.shape(bc))

    # Grad of: c = a + b
    _ba2 = tangent.unbroadcast(bc, a)
    _bb = tangent.unbroadcast(bc, b)
    ba = _ba2
    bb = _bb

    # Grad of: b = x * a
    _bx3 = tangent.unbroadcast(bb * a, x)
    _ba = tangent.unbroadcast(bb * x, a)
    bx = _bx3
    ba = tangent.add_grad(ba, _ba)

    # Grad of: a = x * x
    _bx = tangent.unbroadcast(ba * x, x)
    _bx2 = tangent.unbroadcast(ba * x, x)
    bx = tangent.add_grad(bx, _bx)
    bx = tangent.add_grad(bx, _bx2)
    return bx



<function tangent_29d7.dfdx>

In [0]:
def f(x):
  a = x * x
  with tangent.insert_grad_of(x) as dx:
    print("dc/dx = %2.2f" % dx)
    pdb.set_trace()
  b = x * a
  c = a + b
  
  return c

tangent.grad(f, verbose=True)  

def dfdx(x, bc=1.0):
    a = x * x
    b = x * a
    c = a + b
    assert tangent.shapes_match(c, bc
        ), 'Shape mismatch between return value (%s) and seed derivative (%s)' % (
        numpy.shape(c), numpy.shape(bc))

    # Grad of: c = a + b
    _ba2 = tangent.unbroadcast(bc, a)
    _bb = tangent.unbroadcast(bc, b)
    ba = _ba2
    bb = _bb

    # Grad of: b = x * a
    _bx3 = tangent.unbroadcast(bb * a, x)
    _ba = tangent.unbroadcast(bb * x, a)
    bx = _bx3
    ba = tangent.add_grad(ba, _ba)

    # Inserted code
    print 'dc/dx = %2.2f' % bx
    pdb.set_trace()

    # Grad of: a = x * x
    _bx = tangent.unbroadcast(ba * x, x)
    _bx2 = tangent.unbroadcast(ba * x, x)
    bx = tangent.add_grad(bx, _bx)
    bx = tangent.add_grad(bx, _bx2)
    return bx



<function tangent_9535.dfdx>

# Forward mode

In [0]:
import tangent

def f(x):
  a = x * x
  b = x * a
  c = a + b
  return c

forward_df = tangent.jvp(f, verbose=True)

def _tftx(x, dx):
    if not tangent.shapes_match(x, dx):
        raise ValueError(
            'Shape mismatch between argument value (%s) and seed derivative (%s)'
             % (numpy.shape(x), numpy.shape(dx)))

    # Primal and tangent of: a = x * x
    tmp = x * x
    da = dx * x + x * dx
    a = tmp

    # Primal and tangent of: b = x * a
    db = dx * a + x * da

    # Primal and tangent of: c = a + b
    dc = da + db
    return dc

